<a href="https://colab.research.google.com/github/krypton08rises/Atari-with-reinforcement-learning/blob/master/stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## lets say I have a list of all the stocks im interested in, start with one, let's say apple, most complicated one, we'll have a lot of news articles and blogs
## We find all the information related to the stock on news websites, involves scrapping, we can do scrapping after this
## Then we make an ML model to see the change in stock price relative to our sentiment analysis.
## I am thinking of  analysing the stock to see % day change in closing price, set a threshold, and flag all days where the % day change was higher than a certain amount and the correlation with the sentiment analysis
## Later we can also see what politicians are trading these stocks, I remember there was a website about it

In [2]:
pip install -q newsapi-python

In [3]:
import re
import math
import requests
import pandas as pd
import yfinance as yf
from datetime import date

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [4]:
def get_index_data(index_ticker='^FTSE'):

  df = yf.download(index_ticker,
  start='2023-01-01',
  end=date.today(),
  progress=False)
  df.Close = df.Close.apply(lambda x: math.floor(x))
  return df

In [5]:
df = get_index_data('AAPL')

# Dataset Class


In [6]:
# {'source': {'id': 'the-verge', 'name': 'The Verge'},
  # 'author': 'Sheena Vasani',
  # 'title': 'The latest iPad Air has dropped to an all-time low',
  # 'description': 'The 2022 iPad Air is on sale starting at $449.99, its lowest price to date. You can also grab the Aqara Smart Lock U100 for $100 off and save on Google’s Pixel 8 phone.',
  # 'url': 'https://www.theverge.com/2024/1/30/24054324/apple-ipad-air-m1-aqara-smart-lock-u100-google-pixel-8-deal-sale',
  # 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/Su_yrB-Qr0KRjoF75opyMlCSkXE=/0x0:2040x1360/1200x628/filters:focal(1020x680:1021x681)/cdn.vox-cdn.com/uploads/chorus_asset/file/23318501/akrales_220314_5071_0026.jpg',
  # 'publishedAt': '2024-01-30T15:36:53Z',
  # 'content': 'The latest iPad Air has dropped to an all-time low\r\nThe latest iPad Air has dropped to an all-time low\r\n / Plus, we found deals on the Google Pixel 8 and our favorite Matter-ready smart lock, the Aqa… [+5058 chars]'},

class News_Dataset():
  def __init__(self):
    self.df = pd.DataFrame(columns=['Date', 'Publisher', 'Title', 'Text'], index=None) ## columns to add : Publisher_relevance, Title Sentiment, article_sentiment

In [7]:
news = News_Dataset()

#FT news articles

In [8]:
import requests
from bs4 import BeautifulSoup
news_titles=[]

for page in range(1,6):
    url="https://www.ft.com/world?page={}".format(page)
    result=requests.get(url)
    results=result.content
    soup=BeautifulSoup(results, "lxml")
    for title in soup.findAll("div",{"class":"o-teaser__heading"}):
        titles=title.find(string=True)
        news_titles.append(titles)

print(news_titles)

['Geert Wilders struggles to find coalition partners in the Netherlands', 'Argentina’s congress deals setback to Javier Milei’s reform bill', 'Biden blames Trump for sinking US deal to fund Ukraine war effort', 'McKinsey and BCG warn staff face jail if they reveal Saudi work', 'Zambia raises hopes it will complete long-delayed debt restructuring', 'Qatar says Hamas ‘positive’ on proposed deal to release Gaza hostages', 'Isabel Schnabel: ‘The last mile of disinflation may be the most difficult one’', 'Israel’s business ties to UAE tested by Gaza war fury', 'Trump gives US Supreme Court its biggest election test since Bush vs Gore ', 'The west shouldn’t beg for its reputation', 'Who are the candidates vying to be Trump’s running mate?', 'Inside Mariupol: Russia’s new Potemkin village', 'War in Ukraine: economy and trade', 'The Israel-Hamas war in maps: latest updates', 'Tracking Ukraine’s counteroffensive against Russia in maps', 'Global inflation tracker: see how your country compares o

In [9]:
for title in news_titles:
  if 'apple' in title.lower():
    print(title)

# News API

In [10]:
from newsapi import NewsApiClient

In [11]:
##BBC APi key->01c51ab5526c4853973b78dc3af55c0c

In [12]:
## At the moment, just using the apple keyword, in the future, maybe add other keywords like tim cook, maybe companies that apple owns etc.

In [13]:
newsapi = NewsApiClient(api_key='01c51ab5526c4853973b78dc3af55c0c')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='apple',
                                          sources='bbc-news,the-verge',
                                          # category='business',
                                          language='en',
                                          # country='us'
                                          )

# /v2/everything
all_articles = newsapi.get_everything(q='apple',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2024-01-06',
                                      to='2024-01-31',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# /v2/top-headlines/sources
sources = newsapi.get_sources()

In [14]:
all_articles['articles'][0]['url']

'https://www.theverge.com/2024/1/18/24042756/iowa-tiktok-lawsuit-app-store-rating'

# Getting articles from URL


In [15]:
import urllib

In [16]:
link = all_articles['articles'][0]['url']
f = requests.get(link)

In [17]:
article_content = f.text## get_article_body(f.text)

In [18]:
print(link)
print(article_content)

https://www.theverge.com/2024/1/18/24042756/iowa-tiktok-lawsuit-app-store-rating
<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="twitter:card" content="summary_large_image"/><meta name="twitter:site" content="@verge"/><meta property="fb:app_id" content="549923288395304"/><meta property="og:site_name" content="The Verge"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><meta name="apple-mobile-web-app-title" content="Verge"/><meta name="google-site-verification" content="IucFf_TKtbFFH8_YeFyEteQIwYPdANM1R46_U9DpAr4"/><link rel="alternate" type="application/rss+xml" title="The Verge" href="/rss/index.xml"/><title>Iowa sues TikTok for setting its App Store age rating too low - The Verge</title><meta name="robots" content="index,follow,max-image-preview:large"/><meta name="description" content="The state of Iowa is suing TikTok over claims the app is misleading parents about the presence of sexual content, drugs, alcohol, and pr

# Regex for article Body

In [19]:
def cleanhtml(raw_html):
  CLEANR = re.compile('<.*?>')
  cleantext = re.sub(CLEANR, '', raw_html)
  cleanertext = re.sub('\s+', ' ', cleantext)
  cleanesttext = re.sub(r"&#\d+;", '', cleanertext)
  return cleanesttext
print(cleanhtml(article_content))
# print(re.sub('(<.*>)|(</.*>)', '',contents))### try between speakable-summary and)


Iowa sues TikTok for setting its App Store age rating too low - The Verge{"@context":"http://schema.org/","@type":"NewsArticle","headline":"Iowa sues TikTok for setting its App Store age rating too low","description":"The state of Iowa is suing TikTok over claims the app is misleading parents about the presence of sexual content, drugs, alcohol, and profanity on the app.","datePublished":"2024-01-18T15:08:56.258Z","dateModified":"2024-01-18T15:08:56.258Z","thumbnailUrl":"https://cdn.vox-cdn.com/thumbor/U1KTEe7bC91jh9qgdZ_cDbRtmQg=/0x0:2040x1360/1400x788/filters:focal(1020x680:1021x681)/cdn.vox-cdn.com/uploads/chorus_asset/file/23951408/STK051_VRG_Illo_N_Barclay_4_tiktok.jpg","author":[{"@type":"Person","name":"Emma Roth","url":"https://www.theverge.com/authors/emma-roth"}],"publisher":{"@type":"Organization","name":"The Verge","logo":{"@type":"ImageObject","url":"https://cdn.vox-cdn.com/uploads/chorus_asset/file/24015294/verge_duet_google_news.png","width":250,"height":50}},"image":[{"

In [20]:
#  {'source': {'id': 'the-verge', 'name': 'The Verge'},
#   'author': 'Sheena Vasani',
#   'title': 'The latest iPad Air has dropped to an all-time low',
#   'description': 'The 2022 iPad Air is on sale starting at $449.99, its lowest price to date. You can also grab the Aqara Smart Lock U100 for $100 off and save on Google’s Pixel 8 phone.',
#   'url': 'https://www.theverge.com/2024/1/30/24054324/apple-ipad-air-m1-aqara-smart-lock-u100-google-pixel-8-deal-sale',
#   'urlToImage': 'https://cdn.vox-cdn.com/thumbor/Su_yrB-Qr0KRjoF75opyMlCSkXE=/0x0:2040x1360/1200x628/filters:focal(1020x680:1021x681)/cdn.vox-cdn.com/uploads/chorus_asset/file/23318501/akrales_220314_5071_0026.jpg',
#   'publishedAt': '2024-01-30T15:36:53Z',
#   'content': 'The latest iPad Air has dropped to an all-time low\r\nThe latest iPad Air has dropped to an all-time low\r\n / Plus, we found deals on the Google Pixel 8 and our favorite Matter-ready smart lock, the Aqa… [+5058 chars]'},

for idx, article in enumerate(all_articles['articles']):
  if idx==10:
    break
  f = requests.get(article['url'])
  article_content = f.text
  clean_content = cleanhtml(article_content)
  # print(clean_content)
  news.df.loc[len(news.df)] = [article['publishedAt'], article['source']['name'], article['title'], clean_content]
